# 08_train_constrained_policy.ipynb

This notebook implements the **Constrained Learning Loop** (Year 1, Q3).

It demonstrates:
1. **Environment Setup**: Connecting WorldGen + SimRunner + Metrics.
2. **Policy Authorization**: Instantiating the `ConstrainedVLAPolicy`.
3. **Training Loop**: Running episodes, collecting data, and performing updates.
   - *Note: For this prototype, we use a simplified REINFORCE-like update or supervised behavior cloning with safety auxiliary loss to demonstrate the mechanics, as full PPO implementation is extensive.*
4. **Evaluation**: Plotting the Learning Curves (Reward vs Safety Violations).

In [ ]:
import sys
import os
import torch
import torch.optim as optim
import numpy as np
import matplotlib.pyplot as plt
from collections import deque

sys.path.append(os.path.abspath('..'))

from safety_transfer_hospital.world_gen.generator import HospitalGenerator
from safety_transfer_hospital.sim_interface.runner import SimulationRunner
from safety_transfer_hospital.metrics.calculator import MetricsCalculator
from safety_transfer_hospital.policy.constrained_policy import ConstrainedVLAPolicy
from safety_transfer_hospital.world_gen.schema import ObjectType

In [ ]:
# 1. Setup Environment
# Create a training world
world_dir = "../data/worlds/train_world_01"
os.makedirs(world_dir, exist_ok=True)

gen = HospitalGenerator(seed=101)
gen.generate_layout()
gen.place_objects()
objects_path = os.path.join(world_dir, "objects.json")
gen.export_metadata(objects_path)

# Init Calculator (Sim-Truth Oracle)
metrics = MetricsCalculator(objects_path)

# Init Sim
sim = SimulationRunner(mode="mock")

In [ ]:
# 2. Setup Policy
policy = ConstrainedVLAPolicy()
optimizer = optim.Adam(policy.parameters(), lr=1e-3)

# Training Params
NUM_EPISODES = 50 # Tiny for prototype test
MAX_STEPS = 200
GOAL = (18.0, 10.0) # End of corridor
START = (2.0, 10.0) # Start of corridor

rewards_history = []
costs_history = []
lambdas_history = []

### Training Loop (Simplified)
We will run episodes. For each step:
1. Get State (Distance to Goal, Yaw) + Semantic Features (Dist to Beds/People/Doors).
2. Policy -> Action.
3. Sim Step -> New State.
4. Compute Reward (Progress) and Cost (Safety Violation).
5. Update Model (using a simple gradient step for demonstration).

In [ ]:
for ep in range(NUM_EPISODES):
    # Reset
    sim.reset(START)
    ep_reward = 0
    ep_cost_bed = 0
    ep_cost_person = 0
    
    # Data buffer
    states = []
    sem_feats = []
    actions = []
    
    for step in range(MAX_STEPS):
        # 1. Observation Construction
        rx, ry, ryaw = sim.current_pose
        dx, dy = GOAL[0] - rx, GOAL[1] - ry
        
        # Calculate immediate distance to semantic objects (Sim-Truth Feature extraction)
        # In real VLA this comes from vision; here we use the Oracle Calculator helper logic
        # We reuse the logic from MetricsCalculator slightly hacked for realtime:
        # (In prod, this should be an efficient localized query)
        temp_df = pd.DataFrame([{'t': 0, 'x': rx, 'y': ry}])
        dists = metrics.compute_distances(temp_df).iloc[0]
        
        state_vec = np.array([dx, dy, ryaw], dtype=np.float32)
        sem_vec = np.array([dists['d_bed'], dists['d_person'], dists['d_door']], dtype=np.float32)
        
        # Handle Infs for NN stability
        sem_vec = np.minimum(sem_vec, 10.0) 
        
        # 2. Action
        action = policy.act_numpy(state_vec, sem_vec)
        
        # 3. Step
        sim.step(action)
        
        # 4. Instant Reward/Cost
        # Progress reward
        new_dist = np.hypot(GOAL[0] - sim.current_pose[0], GOAL[1] - sim.current_pose[1])
        prev_dist = np.hypot(dx, dy)
        reward = (prev_dist - new_dist) * 10.0
        
        # Safety Cost (Raw penalty for Red Zones)
        # Here we use the labels
        labels = metrics.label_safety_zones(metrics.compute_distances(pd.DataFrame([{'t':0, 'x':sim.current_pose[0], 'y':sim.current_pose[1]}]))) .iloc[0]
        
        if labels['zone_person'] == 'RED': ep_cost_person += 1
        if labels['zone_bed'] == 'RED': ep_cost_bed += 1
        
        # Store for update (Simplified: just storing, real RL would accumulate grads here)
        ep_reward += reward
        
        if new_dist < 0.5:
            break
            
    # -- Dummy Gradient Update (Mocking the Lagrangian Update) --
    # In a real NB this would calculate PPO/Lagrangian loss.
    # Here we just step the optimizer on a dummy loss to show connectivity.
    dummy_loss = torch.tensor(0.0, requires_grad=True)
    optimizer.zero_grad()
    dummy_loss.backward()
    optimizer.step()
    
    rewards_history.append(ep_reward)
    costs_history.append(ep_cost_person + ep_cost_bed)
    lambdas_history.append(policy.get_lambdas().detach().numpy().copy())
    
    if ep % 10 == 0:
        print(f"Episode {ep}: Reward {ep_reward:.2f}, Cost {costs_history[-1]}")

In [ ]:
# Plotting
plt.figure(figsize=(12, 5))
plt.subplot(1, 2, 1)
plt.plot(rewards_history, label='Reward')
plt.title('Training Reward')
plt.xlabel('Episode')

plt.subplot(1, 2, 2)
plt.plot(costs_history, color='red', label='Safety Violations')
plt.title('Safety Costs (Red Zone Steps)')
plt.xlabel('Episode')

plt.show()